# New York Police Department : Stop & Frisk data set
The aim of this assignment is to download and inspect the Stop, Question and Frisk Data released by the New York Police Department every year.
We will use the data available on this link : https://www1.nyc.gov/site/nypd/stats/reports-analysis/stopfrisk.page
We will use data from 2003 until 2011.

We start by retrieving the data for the year 2003, directly from the source.

In [ ]:
import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
url = urlopen("https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2003-csv.zip")
#https://stackoverflow.com/questions/40852480/python-loading-zip-file-stored-in-csv-from-web
#Download Zipfile and create pandas DataFrame
zipfile = ZipFile(BytesIO(url.read()))
df_2003 = pd.read_csv(zipfile.open('2003.csv'),error_bad_lines=False, index_col=False, dtype='unicode',encoding ='latin1')
df_2003

Next, we extract the data for all the remaining years and concatenate them into one dataframe. Note that the DF contains more than 4 million rows, it might result in a *time out* request. 

In [ ]:

file_urls=['https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2003-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2004-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2005-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2006-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2007-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2008-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2009-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2010-csv.zip',
'https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/sqf-2011-csv.zip']


for x in file_urls:
    urls = urlopen(x)
    zipfile = ZipFile(BytesIO(urls.read()))
    fname = zipfile.namelist()[0]
    df = pd.read_csv(zipfile.open(fname), error_bad_lines=False, index_col=False, dtype='unicode',encoding ='latin1')
    zipfile.close()
    
    # let me know how much it is completed.
    print(f'{fname} Completed')
  


In [ ]:
#making the full dataframe
full_df = pd.concat([pd.read_csv(x,error_bad_lines=False, index_col=False, dtype='unicode',encoding ='latin1') for x in file_urls])
df = full_df
df.reset_index(level =0,drop=True,inplace=True)
df

# Understanding the data

Record layouts and code books for each year's data are recorded in separate Excel spreadsheet format files. A zip archive containing all years' spreadsheets can be downloaded from the NYPD website. In what follows, you are going to retrieve the code books from this archive and create a legend for your database.

Given that the data cover a 8-year period, it seems reasonable to assume that they have undergone some changes, i.e. that the columns are not exactly the same throughout the years. To check if this is the case, let's start by taking a look at the code books for 2003 and 2011, the first and last year in our considered period.

We write code to read the code book spreadsheets for the years 2003 and 2011 and store them in pandas DataFrame objects called legend_2003 and legend_2011, respectively. These DataFrames should contain only two columns: Variable (the names of the columns) and Label (the descriptions for the different column names).

In [ ]:
#first for 2003
url = urlopen("https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/SQF-File-Documentation.zip")
zipfile = ZipFile(BytesIO(url.read()))
legend_2003 = pd.read_excel(zipfile.open('2003 SQF File Spec.xlsx'),index_col=False, dtype='unicode',skiprows=3,usecols = ['Variable','Label'])

#then for 2011
legend_2011 = pd.read_excel(zipfile.open('2011 SQF File Spec.xlsx'),index_col=False, dtype='unicode',skiprows=3,usecols=['Variable','Label'])

legend_2003

The lengths of both DataFrames are different. This means that there are slight variations in the columns for each year. As a result, you will need to import the code books for all the different years to create your legend and ensure that all column names are properly documented.
Once again, the files are opened direclty from the source.


In [ ]:
import fnmatch
url = urlopen("https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/SQF-File-Documentation.zip")
with ZipFile(BytesIO(url.read())) as zipfiles:

    file_list = zipfiles.namelist()[1:10]
    
    #get only the excel files
    excel_files = fnmatch.filter(file_list, "*.xlsx")
    
    #iterate with a list comprehension to get the individual dataframes
    legend = [pd.read_excel(zipfiles.open(file_name),index_col=False, dtype='unicode',skiprows=3,usecols=['Variable','Label'])for file_name in excel_files]

#combine into one dataframe
legend = pd.concat(legend)
#drop duplicate values
legend = legend.drop_duplicates()
legend

You'll notice that the columns included in the documentation differ from those in the df DataFrame you created above: all the column names included in legend are also included in df, but the opposite is not true. This is because there are several spelling mistakes in the databases we imported at the beginning.
Therefore, we retrieve the typose

In [ ]:
typos = []

#loop through column elements in df
for x in df.columns:
    if x in legend.values:
        None
    else:
        typos.append(x)
typos


Dedicate a few minutes to revise the list of columns in legend. Identify the correct column names for each of the entries in typos. Notice that the differences are mostly due to spelling mistakes, or uppercase vs. lowercase issues. There's one single case where the correspondence is not clear: 'wepfound'. You may need to check the documentation once again to identify the right column name for this value.

We now write the code to modify the column names in df so that those column names included in typos are all renamed to match those in legend. Once you are done renaming the columns, merge the columns with the same names to eliminate the duplicates.

In [ ]:
import numpy as np
#create dictionary with the right corresponding keys and values to rename the columns
df = df.rename(columns={'dettypcm' :'dettypCM','linecm': 'lineCM','detailcm': 'detailCM',
                          'wepfound': 'othrweap','dettyp_c': 'dettypCM','details_':'detailCM',
                          'detail1_': 'detailCM','rescod': 'rescode', 'premtyp': 'premtype',
                          'prenam': 'premname', 'adrnum': 'addrnum', 'strname': 'stname', 'strintr':'stinter','adrpct':'addrpct'})
#find duplicate values and put them in a list
my_list=[]
for cols in df.columns:
    my_list.append(cols)
    
#iterate through the duplicate list to print only the unique duplicate values
duplicates = []
for i in range(0, len(my_list)):    
    for j in range(i+1, len(my_list)):    
        if(my_list[i] == my_list[j]):    
            duplicates.append(my_list[j])
#list of unique duplicate values            
duplicates_clean = np.unique(duplicates)
print(duplicates_clean)

In [ ]:
#find column index values from the unique duplicate values list
for x in duplicates_clean:
    index_no = df.columns.get_loc(x)
    index = (np.where(index_no)[0])
    index_list = index.tolist()
    print(index_list)


In [ ]:
#let's create DataFrames for each group of index values returned from the above numpy array
#then let's concatenate them, combine them and convert them to a DataFrame

#addrnum
data_addrnum = [df.iloc[:,94], df.iloc[:,118]]
df_2 = pd.concat(data_addrnum,axis=1)
df_2 = df_2.iloc[:,0].combine_first(df_2.iloc[:,1])
df_2 = df_2.to_frame().reset_index(drop=True)

#detailCM
data_detailCM = [df.iloc[:,110], df.iloc[:,113], df.iloc[:,114]]
df_3 = pd.concat(data_detailCM,axis=1)
df_3 = df_3.iloc[:,0].combine_first(df_3.iloc[:,1]).combine_first(df_3.iloc[:,2])
df_3 = df_3.to_frame().reset_index(drop=True)


#dettypCM
data_dettypCM = [df.iloc[:,108],df.iloc[:,112]]
df_4 = pd.concat(data_dettypCM,axis=1)
df_4 = df_4.iloc[:,0].combine_first(df_4.iloc[:,1])
df_4 = df_4.to_frame().reset_index(drop=True)


#othrweap
data_othrweap = [df.iloc[:,31],df.iloc[:,111]]
df_5 = pd.concat(data_othrweap,axis=1)
df_5 = df_5.iloc[:,0].combine_first(df_5.iloc[:,1])
df_5 = df_5.to_frame().reset_index(drop=True)

#premname
data_premname = [df.iloc[:,93],df.iloc[:,117]]
df_6 = pd.concat(data_premname,axis=1)
df_6 = df_6.iloc[:,0].combine_first(df_6.iloc[:,1])
df_6 = df_6.to_frame().reset_index(drop=True)

#premtype
data_premtype = [df.iloc[:,92],df.iloc[:,116]]
df_7 = pd.concat(data_premtype,axis=1)
df_7 = df_7.iloc[:,0].combine_first(df_7.iloc[:,1])
df_7 = df_7.to_frame().reset_index(drop=True)

#rescode
data_rescode = [df.iloc[:,91],df.iloc[:,115]]
df_8 = pd.concat(data_rescode,axis=1)
df_8 = df_8.iloc[:,0].combine_first(df_8.iloc[:,1])
df_8 = df_8.to_frame().reset_index(drop=True)

#stname
data_stname = [df.iloc[:,95],df.iloc[:,119]]
df_9 = pd.concat(data_stname,axis=1)
df_9 = df_9.iloc[:,0].combine_first(df_9.iloc[:,1])
df_9 = df_9.to_frame().reset_index(drop=True)

#stinter
data_stinter = [df.iloc[:,96],df.iloc[:,120]]
df_10 = pd.concat(data_stinter,axis=1)
df_10 = df_10.iloc[:,0].combine_first(df_10.iloc[:,1])
df_10 = df_10.to_frame().reset_index(drop=True)

#addrpct
data_addrpct = [df.iloc[:,102],df.iloc[:,121]]
df_11 = pd.concat(data_addrpct,axis=1)
df_11 = df_11.iloc[:,0].combine_first(df_11.iloc[:,1])
df_11 = df_11.to_frame().reset_index(drop=True)

#drop the duplicate column names and concatenate all of the new DataFrames (form df_2 to df_9 with the oririnal df)
df = df.drop(columns=['addrnum', 'othrweap', 'detailCM', 'dettypCM', 'premname','premtype',
 'rescode', 'stname','stinter','addrpct'],axis=1)
df=  pd.concat([df,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11], axis=1)
df = df.reset_index(drop=True)
df

In [ ]:
#let's just make sure that our DataFrame does not contain anymore duplicate column names
my_list_2=[]
for cols in df.columns:
    my_list_2.append(cols)
duplicates = []
for i in range(0, len(my_list_2)):    
    for j in range(i+1, len(my_list_2)):    
        if(my_list_2[i] == my_list_2[j]):    
            duplicates.append(my_list_2[j])
#list of unique duplicate values   which should be empty          
duplicates_clean = np.unique(duplicates)
print(duplicates_clean)

Now that we have cleaned the column names and merged the data, let's remove all unnecesary information. The list shown below contains the descriptions for the variables we are interesting in.

In [ ]:
information = ["YEAR OF STOP (CCYY)", "PRECINCT OF STOP (FROM 1 TO 123)", "DATE OF STOP (MM-DD-YYYY)", 
               "CRIME SUSPECTED", "WAS AN ARREST MADE ?", "PERIOD OF STOP (MMM)",
               "WAS A SUMMONS ISSUED ?", "WAS A PISTOL FOUND ON SUSPECT ?", "WAS A RIFLE FOUND ON SUSPECT ?", 
               "WAS AN ASSAULT WEAPON FOUND ON SUSPECT ?", "WAS A KNIFE OR CUTTING INSTRUMENT FOUND ON SUSPECT ?",
               "WAS A MACHINE GUN FOUND ON SUSPECT ?","WAS ANOTHER TYPE OF WEAPON FOUND ON SUSPECT", 
               "PHYSICAL FORCE USED BY OFFICER - HANDS", "PHYSICAL FORCE USED BY OFFICER - SUSPECT AGAINST WALL",
               "PHYSICAL FORCE USED BY OFFICER - SUSPECT ON GROUND", "PHYSICAL FORCE USED BY OFFICER - WEAPON DRAWN",
               "PHYSICAL FORCE USED BY OFFICER - WEAPON POINTED", "PHYSICAL FORCE USED BY OFFICER - BATON",
               "PHYSICAL FORCE USED BY OFFICER - HANDCUFFS", "PHYSICAL FORCE USED BY OFFICER - PEPPER SPRAY",
               "PHYSICAL FORCE USED BY OFFICER - OTHER", "SUSPECT'S RACE", "SUSPECT'S SEX"]

Then, we write the code to identify the columns that contain the information above and remove all the other columns from the df DataFrame. Don't modify neither the name of the Dataframe nor any of the given column names when doing so.

In [ ]:
variable_info = []

for l in information:
    variable_info.append(legend.loc[legend['Label'] == l, 'Variable'].values[0])
    
df = df[variable_info]

df

# Enriching the data

The column pct encodes the precinct where each stop took place. Police units based in these precincts are responsible for safety and law enforcement within a designated geographic area. There are a total of 77 precincts in the city of New York, each identified with a number from 1 to 123. These precincts can be further aggregated into the 5 major boroughs of New York: Bronx, Brooklyn, Manhattan, Queens and Staten Island. Let's enrich the data by adding the borough data so that we can later visualize it using different levels of granularity.

To do so, you will need to look for the correspondences between boroughs and police precincts, and add this information to your database. Before doing so, however, you will need to make sure that the pct column is properly encoded.

We now write the code to standardize the values in the pct column by converting all entries to integers. Don't modify the name of the column when doing so and don't add any new columns to the DataFrame either.

In [ ]:
#convert to numeric value
df.loc[:,'pct'] = pd.to_numeric(df.loc[:,'pct'], errors = 'coerce', downcast = 'integer')
#drop the NaN values
df.dropna(subset = ['pct'], inplace = True)
#convert all to integers
df['pct'] = pd.to_numeric(df['pct'], downcast = 'integer')


Now that the pct column is properly encoded, let's include the information regarding the borough.

In [ ]:
#Create a list of condition to identify the tiers

conditions = [
    (df['pct'] <= 39), #Manhattan
    (df['pct'] >39 ) & (df['pct'] <= 59), #Bronx
    (df['pct'] > 59) & (df['pct'] <= 99), #Brooklyn
    (df['pct'] > 99) & (df['pct']<=119), #Queens
    (df['pct'] > 120) & (df['pct']<=123), #Stated Island
    df['pct']>123 #Everything else
    ]
values = ['Manhattan', 'Bronx', 'Brooklyn', 'Queens', ' Staten Island',None]

df['borough'] = np.select(conditions, values)

#drop NA values
df = df.dropna(subset= ['borough'])

#print df to check that number of rows has changed
df

Let's further filter down these data. Some of the columns store related information. For example, lots of the columns refer to PHYSICAL FORCE USED BY OFFICER. Since we are not interested in the specific form of force used, but rather on the existence of such force, let's aggregate the information for these columns.

The same thing can be said for the set of columns that contain the phrase WAS A - FOUND ON SUSPECT ?. Once again, we are not interested in the actual type of weapon found, but on whether a weapon was indeed found on the suspect. Hence, let's aggregate all these data into a single column.

Finally, let's also aggregate the data about whether an arrest was made. We'll consider that an arrest took place if an arrest was actually made or if a summons was issued, or both.

Therefore, we write the code to create new columns called physical_force, weapon and arrest. These columns should take the value 1 in int form if any of the related columns are encoded as "Y" and 0 in int form otherwise.

In [ ]:
#create for phyiscal force
df['physical_force'] = np.where(df['pf_hands'].isin(['Y']) | df['pf_wall'].isin(['Y']) | df['pf_grnd'].isin(['Y']) | df['pf_drwep'].isin(['Y']) | df['pf_ptwep'].isin(['Y']) | df['pf_baton'].isin(['Y']) | df['pf_hcuff'].isin(['Y']) | df['pf_pepsp'].isin(['Y']) | df['pf_other'].isin(['Y']), 1,0)

#create for weapon
df['weapon'] = np.where(df['pistol'].isin(['Y']) | df['riflshot'].isin(['Y']) | df['knifcuti'].isin(['Y']) | df['machgun'].isin(['Y']) | df['othrweap'].isin(['Y']), 1,0)

#create for arrest
df['arrest'] = np.where(df['arstmade'].isin(['Y']) | df['sumissue'].isin(['Y']), 1,0)

We are now going to focus on the race column. This column encodes critical information for our use case. You can find more information regarding the different letter codes in the documentation provided by NYPD. We will begin by changing the name of the column to ethnicity. Ideally, we would prefer to keep the original values for this variable. 

Therefore, we write the code to rename the race column as ethnicity. Then write the code to modify the values of your ethnicity column. This column should take the value 'Black' if the original record was encoded as Black or Black-Hispanic. It should take the value 'White' if the original record was encoded as White or White-Hispanic. It should take the value 'Asian' if the original record was encoded as Asian/Pacific Islander. It should take the value 'Other' if the original record was encoded as American Indian/Alaskan native, Other, Unknown or Not listed.

In [ ]:
df = df.rename(columns={'race' :'ethnicity'})


df['ethnicity'] = np.where((df.ethnicity == 'B'),'Black',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'P'),'Black',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'I'),'Other',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'Q'),'White',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'W'),'White',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'A'),'Asian',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == ' '),'Other',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'X'),'Other',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'Z'),'Other',df.ethnicity)
df['ethnicity'] = np.where((df.ethnicity == 'U'),'Other',df.ethnicity) 
df

Finally, let's remove redundant information.

In [2]:
df = df[['year','pct','datestop','perstop','crimsusp','ethnicity','sex','borough','physical_force','weapon','arrest']]
df

NameError: name 'df' is not defined